### 1.ConversationBufferMemory

In [9]:
!python -m venv venvnb2 || source venvnb2\Scripts\activate
!pip install -qU langchain-core langchain-ollama langchain-community

Inicializamos el model con `phi3` local

In [28]:
from langchain_ollama.llms import OllamaLLM

# Inicializa el modelo Ollama LLM
llm = OllamaLLM(model="llama3.2")

In [11]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

Hay varias maneras de añadir mensajes a nuestra memoria. Usando el método save_context, podemos añadir una consulta del usuario (mediante la clave de entrada) y la respuesta de la IA (mediante la clave de salida). Para crear la siguiente conversación:

- Usuario: Hola, me llamo Javier.
- IA: ¡Hola Javier! Encantado de conocerte, soy un modelo de IA llamado Zeta.
- Usuario: Estoy investigando los distintos tipos de memoria conversacional en LangChain.
- IA: Qué interesante, ¿cuáles has estado explorando?
- Usuario: He estado viendo ConversationBufferMemory y ConversationBufferWindowMemory.
- IA: Suena bien. ¿Cuál es la diferencia entre ellos?
- Usuario: La memoria tipo buffer guarda toda la conversación completa, ¿cierto?
- IA: Exacto. ¿Y qué hace la memoria tipo buffer window?
- Usuario: Esa guarda solo los últimos k mensajes y descarta el resto.
- IA: ¡Muy útil para mantener el contexto sin saturar el modelo!

Entonces:

In [16]:
memory.save_context(
    {"input": "Hola, me llamo Javier."},
    {"output": "¡Hola Javier! Encantado de conocerte, soy un modelo de IA llamado Zeta."}
)
memory.save_context(
    {"input": "Estoy investigando los distintos tipos de memoria conversacional en LangChain."},
    {"output": "Qué interesante, ¿cuáles has estado explorando?"}
)
memory.save_context(
    {"input": "He estado viendo ConversationBufferMemory y ConversationBufferWindowMemory."},
    {"output": "Suena bien. ¿Cuál es la diferencia entre ellos?"}
)
memory.save_context(
    {"input": "La memoria tipo buffer guarda toda la conversación completa, ¿cierto?"},
    {"output": "Exacto. ¿Y qué hace la memoria tipo buffer window?"}
)
memory.save_context(
    {"input": "Esa guarda solo los últimos k mensajes y descarta el resto."},
    {"output": "¡Muy útil para mantener el contexto sin saturar el modelo!"}
)

In [17]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hola, me llamo Javier.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='¡Hola Javier! Encantado de conocerte, soy un modelo de IA llamado Zeta.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Estoy investigando los distintos tipos de memoria conversacional en LangChain.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Qué interesante, ¿cuáles has estado explorando?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='He estado viendo ConversationBufferMemory y ConversationBufferWindowMemory.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Suena bien. ¿Cuál es la diferencia entre ellos?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='La memoria tipo buffer guarda toda la conversación completa, ¿cierto?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Exacto. ¿Y qué hace la memoria tipo buffer window?', additional_

Con esto, ya tenemos nuestra memoria tipo buffer lista.
Antes de conectarla al LLM, revisemos otra forma de agregar mensajes a la memoria.

En lugar de guardar el contexto completo de una conversación, esta alternativa permite añadir los mensajes de manera individual, tanto del usuario como de la IA, utilizando los métodos add_user_message y add_ai_message.

Si quisiéramos replicar lo que hicimos anteriormente, lo haríamos así:

In [19]:
memory = ConversationBufferMemory(return_messages=True)

memory.chat_memory.add_user_message("Hola, me llamo Javier.")
memory.chat_memory.add_ai_message("¡Hola Javier! Encantado de conocerte, soy un modelo de IA llamado Zeta.")
memory.chat_memory.add_user_message("Estoy investigando los distintos tipos de memoria conversacional en LangChain.")
memory.chat_memory.add_ai_message("Qué interesante, ¿cuáles has estado explorando?")
memory.chat_memory.add_user_message("He estado viendo ConversationBufferMemory y ConversationBufferWindowMemory.")
memory.chat_memory.add_ai_message("Suena bien. ¿Cuál es la diferencia entre ellos?")
memory.chat_memory.add_user_message("La memoria tipo buffer guarda toda la conversación completa, ¿cierto?")
memory.chat_memory.add_ai_message("Exacto. ¿Y qué hace la memoria tipo buffer window?")
memory.chat_memory.add_user_message("Esa guarda solo los últimos k mensajes y descarta el resto.")
memory.chat_memory.add_ai_message("¡Muy útil para mantener el contexto sin saturar el modelo!")

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hola, me llamo Javier.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='¡Hola Javier! Encantado de conocerte, soy un modelo de IA llamado Zeta.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Estoy investigando los distintos tipos de memoria conversacional en LangChain.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Qué interesante, ¿cuáles has estado explorando?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='He estado viendo ConversationBufferMemory y ConversationBufferWindowMemory.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Suena bien. ¿Cuál es la diferencia entre ellos?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='La memoria tipo buffer guarda toda la conversación completa, ¿cierto?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Exacto. ¿Y qué hace la memoria tipo buffer window?', additional_

El resultado final es idéntico en ambos casos.  
Para poder conectar esta memoria con nuestro modelo de lenguaje (LLM), necesitamos crear un objeto `ConversationChain`.

Sin embargo, es importante tener en cuenta que esta clase ha quedado obsoleta y ha sido reemplazada por `RunnableWithMessageHistory`, la cual veremos más adelante.

In [30]:
from langchain.chains import ConversationChain

chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [38]:
response = chain.invoke({"input": "de que te estaba hablando? (dímelo de forma puntual y breve)"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hola, me llamo Javier.', additional_kwargs={}, response_metadata={}), AIMessage(content='¡Hola Javier! Encantado de conocerte, soy un modelo de IA llamado Zeta.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Estoy investigando los distintos tipos de memoria conversacional en LangChain.', additional_kwargs={}, response_metadata={}), AIMessage(content='Qué interesante, ¿cuáles has estado explorando?', additional_kwargs={}, response_metadata={}), HumanMessage(content='He estado viendo ConversationBufferMemory y ConversationBufferWindowMemory.', additional_kwargs={}, response_metadata={}), AIMessage(content='Suena bien. ¿Cuál es

In [39]:
print(response['response'])

La conversación anterior se centró en la exploración de tipos de memoria conversacional en LangChain, específicamente ConversationBufferMemory y ConversationBufferWindowMemory. Estabas preguntando sobre las diferencias entre ellos y cómo funcionan.


### ConversationBufferMemory with RunnableWithMessageHistory
Como se advierte, el tipo `ConversationBufferMemory` está a punto de quedar obsoleto. En su lugar, podemos usar la clase `RunnableWithMessageHistory` para implementar la misma funcionalidad.

Al implementar `RunnableWithMessageHistory`, usaremos el **LangChain Expression Language (LCEL)**. Para ello, necesitamos definir nuestra plantilla de mensaje y los componentes LLM. Con el LLM ya nos enfocaremos en `ChatPromptTemplate`.

In [43]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    ChatPromptTemplate
)

system_prompt = "Eres un asistente muy útil llamado Zeta, respondes siempre de forma breve y puntual."

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{query}"),
])

Esta es la forma en la que trabaja LangChain, concatena (a manera de una cadena justamente) cada uno de los componentes que luego llegan a una invocacion.

In [44]:
pipeline = prompt_template | llm

`RunnableWithMessageHistory` requiere que nuestro pipeline esté en un objeto `RunnableWithMessageHistory`.
Este objeto, requiere de `get_session_history`, el cual requiere a su vez de una función que devuelva un objeto ChatMessageHistory basado en un ID de sesión.

Definimos esta función nosotros mismos:

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory

chat_map = {}
def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in chat_map:
        # Si no existe, crea un nuevo historial de chat en memoria
        chat_map[session_id] = InMemoryChatMessageHistory()
    return chat_map[session_id]

También necesitamos indicar a nuestro runnable qué nombre de variable usar para el historial de chat (por ejemplo, history) y cuál para la consulta del usuario (por ejemplo, query).

In [47]:
from langchain_core.runnables.history import RunnableWithMessageHistory

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history"
)

In [48]:
pipeline_with_history.invoke(
    {"query": "hola, me llamo Javier."},
    config={"session_id": "id_123"}
)

'Hola Javier, encantado de conocerte. ¿En qué puedo ayudarte hoy?'

In [49]:
pipeline_with_history.invoke(
    {"query": "Cuál es mi nombre?"},
    config={"session_id": "id_123"}
)

'Hola Javier, tu nombre en nuestra base de datos es "Javier". No tengo información adicional sobre ti.'

### 2.ConversationBufferMemory

In [57]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=4, return_messages=True)

In [58]:
memory.chat_memory.add_user_message("Hola, me llamo Javier.")
memory.chat_memory.add_ai_message("¡Hola Javier! Encantado de conocerte, soy un modelo de IA llamado Zeta.")
memory.chat_memory.add_user_message("Estoy investigando los distintos tipos de memoria conversacional en LangChain.")
memory.chat_memory.add_ai_message("Qué interesante, ¿cuáles has estado explorando?")
memory.chat_memory.add_user_message("He estado viendo ConversationBufferMemory y ConversationBufferWindowMemory.")
memory.chat_memory.add_ai_message("Suena bien. ¿Cuál es la diferencia entre ellos?")
memory.chat_memory.add_user_message("La memoria tipo buffer guarda toda la conversación completa, ¿cierto?")
memory.chat_memory.add_ai_message("Exacto. ¿Y qué hace la memoria tipo buffer window?")
memory.chat_memory.add_user_message("Esa guarda solo los últimos k mensajes y descarta el resto.")
memory.chat_memory.add_ai_message("¡Muy útil para mantener el contexto sin saturar el modelo!")

memory.load_memory_variables({})

{'history': [HumanMessage(content='Estoy investigando los distintos tipos de memoria conversacional en LangChain.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Qué interesante, ¿cuáles has estado explorando?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='He estado viendo ConversationBufferMemory y ConversationBufferWindowMemory.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Suena bien. ¿Cuál es la diferencia entre ellos?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='La memoria tipo buffer guarda toda la conversación completa, ¿cierto?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Exacto. ¿Y qué hace la memoria tipo buffer window?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Esa guarda solo los últimos k mensajes y descarta el resto.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='¡Muy útil para mantener el contexto sin saturar 

In [59]:
chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [60]:
response = chain.invoke({"input": "cual es mi nombre?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Estoy investigando los distintos tipos de memoria conversacional en LangChain.', additional_kwargs={}, response_metadata={}), AIMessage(content='Qué interesante, ¿cuáles has estado explorando?', additional_kwargs={}, response_metadata={}), HumanMessage(content='He estado viendo ConversationBufferMemory y ConversationBufferWindowMemory.', additional_kwargs={}, response_metadata={}), AIMessage(content='Suena bien. ¿Cuál es la diferencia entre ellos?', additional_kwargs={}, response_metadata={}), HumanMessage(content='La memoria tipo buffer guarda toda la conversación completa, ¿cierto?', additional_kwargs={}, response_metadata={}), AIMessage(conten

In [61]:
print(response['response'])

Lo siento, pero no tengo suficiente información sobre ti para conocerte tu nombre. No tenemos un registro de conversaciones previas y no puedo acceder a datos externos que puedan contener información personal. ¿Podrías decirme tu nombre o cómo te gustaría que lo llame?


La razón por la que nuestro LLM ya no recuerda nuestro nombre es porque hemos establecido el parámetro `k` en `4`, lo que significa que solo se almacenan en memoria los últimos mensajes. Como podemos ver arriba, esto no incluye el primer mensaje donde nos presentamos.

Si consideramos que el agente olvida nuestro nombre, podríamos preguntarnos por qué usaríamos este tipo de memoria en lugar de la memoria intermedia estándar. Como ocurre con la mayoría de los procesos en IA, siempre hay que buscar soluciones intermedias. Aquí podemos mantener conversaciones mucho más largas, usar menos tokens y mejorar la latencia, pero esto implica olvidar mensajes no recientes.

In [62]:
chain.invoke({"input": "hablamos de ConversationBufferWindowMemory?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='He estado viendo ConversationBufferMemory y ConversationBufferWindowMemory.', additional_kwargs={}, response_metadata={}), AIMessage(content='Suena bien. ¿Cuál es la diferencia entre ellos?', additional_kwargs={}, response_metadata={}), HumanMessage(content='La memoria tipo buffer guarda toda la conversación completa, ¿cierto?', additional_kwargs={}, response_metadata={}), AIMessage(content='Exacto. ¿Y qué hace la memoria tipo buffer window?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Esa guarda solo los últimos k mensajes y descarta el resto.', additional_kwargs={}, response_metadata={}), AIMessage(content='¡Muy útil par

{'input': 'hablamos de ConversationBufferWindowMemory?',
 'history': [HumanMessage(content='He estado viendo ConversationBufferMemory y ConversationBufferWindowMemory.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Suena bien. ¿Cuál es la diferencia entre ellos?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='La memoria tipo buffer guarda toda la conversación completa, ¿cierto?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Exacto. ¿Y qué hace la memoria tipo buffer window?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Esa guarda solo los últimos k mensajes y descarta el resto.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='¡Muy útil para mantener el contexto sin saturar el modelo!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='cual es mi nombre?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Lo siento, pero no tengo suficiente info

## ConversationBufferWindowMemory with RunnableWithMessageHistory

In [63]:
from pydantic import BaseModel, Field
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage

class BufferWindowMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    k: int = Field(default_factory=int)

    def __init__(self, k: int):
        super().__init__(k=k)
        print(f"Inicializando BufferWindowMessageHistory con k={k}")

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Agrega mensajes al historial, removiendo cualquier mensaje posterior a los ultimos `k` mensajes."""
        self.messages.extend(messages)
        self.messages = self.messages[-self.k:]

    def clear(self) -> None:
        """limpiar historial."""
        self.messages = []

In [64]:
chat_map = {}
def get_chat_history(session_id: str, k: int = 4) -> BufferWindowMessageHistory:
    print(f"get_chat_history llamado con session_id={session_id} y k={k}")
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = BufferWindowMessageHistory(k=k)
    # remove anything beyond the last
    return chat_map[session_id]

In [65]:
from langchain_core.runnables import ConfigurableFieldSpec

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="El ID de sesión a usarse en el historial de chat",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="k",
            annotation=int,
            name="k",
            description="el número de mensajes a mantener en memoria",
            default=4,
        )
    ]
)

In [67]:
pipeline_with_history.invoke(
    {"query": "hola, me llamo Javier."},
    config={"configurable": {"session_id": "id_k4", "k": 4}}
)

get_chat_history llamado con session_id=id_k4 y k=4


'Disculpa la duplicidad, Javier. ¿En qué puedo ayudarte hoy?'

In [68]:
chat_map["id_k4"].clear()  # limpiar historial

chat_map["id_k4"].add_user_message("Hola, me llamo Javier.")
chat_map["id_k4"].add_ai_message("¡Hola Javier! Encantado de conocerte, soy un modelo de IA llamado Zeta.")
chat_map["id_k4"].add_user_message("Estoy investigando los distintos tipos de memoria conversacional en LangChain.")
chat_map["id_k4"].add_ai_message("Qué interesante, ¿cuáles has estado explorando?")
chat_map["id_k4"].add_user_message("He estado viendo ConversationBufferMemory y ConversationBufferWindowMemory.")
chat_map["id_k4"].add_ai_message("Suena bien. ¿Cuál es la diferencia entre ellos?")
chat_map["id_k4"].add_user_message("La memoria tipo buffer guarda toda la conversación completa, ¿cierto?")
chat_map["id_k4"].add_ai_message("Exacto. ¿Y qué hace la memoria tipo buffer window?")
chat_map["id_k4"].add_user_message("Esa guarda solo los últimos k mensajes y descarta el resto.")
chat_map["id_k4"].add_ai_message("¡Muy útil para mantener el contexto sin saturar el modelo!")

chat_map["id_k4"].messages

[HumanMessage(content='La memoria tipo buffer guarda toda la conversación completa, ¿cierto?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Exacto. ¿Y qué hace la memoria tipo buffer window?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Esa guarda solo los últimos k mensajes y descarta el resto.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='¡Muy útil para mantener el contexto sin saturar el modelo!', additional_kwargs={}, response_metadata={})]

In [69]:
pipeline_with_history.invoke(
    {"query": "cual es mi nombre?"},
    config={"configurable": {"session_id": "id_k4", "k": 4}}
)

get_chat_history llamado con session_id=id_k4 y k=4


'¡Hola! Mi nombre es Zeta, soy tu asistente. No tengo un nombre propio, pero puedo ayudarte en lo que necesites.'

In [70]:
pipeline_with_history.invoke(
    {"query": "hola! soy Javier"},
    config={"session_id": "id_k14", "k": 14}
)

get_chat_history llamado con session_id=id_k14 y k=14
Inicializando BufferWindowMessageHistory con k=14


'Hola Javier, ¿en qué puedo ayudarte hoy?'

Agregamos manualmente el historial

In [74]:
chat_map["id_k14"].add_user_message("Estoy investigando los distintos tipos de memoria conversacional en LangChain.")
chat_map["id_k14"].add_ai_message("Qué interesante, ¿cuáles has estado explorando?")
chat_map["id_k14"].add_user_message("He estado viendo ConversationBufferMemory y ConversationBufferWindowMemory.")
chat_map["id_k14"].add_ai_message("Suena bien. ¿Cuál es la diferencia entre ellos?")
chat_map["id_k14"].add_user_message("La memoria tipo buffer guarda toda la conversación completa, ¿cierto?")
chat_map["id_k14"].add_ai_message("Exacto. ¿Y qué hace la memoria tipo buffer window?")
chat_map["id_k14"].add_user_message("Esa guarda solo los últimos k mensajes y descarta el resto.")
chat_map["id_k14"].add_ai_message("¡Muy útil para mantener el contexto sin saturar el modelo!")
1
chat_map["id_k14"].messages

[HumanMessage(content='hola! soy Javier', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hola Javier, ¿en qué puedo ayudarte hoy?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='cual es mi nombre?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hasta ahora solo sabemos que eres "Javier", pero no tengo información adicional sobre ti. ¿Quieres compartir algo más sobre ti mismo para personalizar mejor nuestra interacción?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Estoy investigando los distintos tipos de memoria conversacional en LangChain.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Qué interesante, ¿cuáles has estado explorando?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='He estado viendo ConversationBufferMemory y ConversationBufferWindowMemory.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Suena bien. ¿Cuál es la diferencia entre 

In [75]:
pipeline_with_history.invoke(
    {"query": "cual es mi nombre?"},
    config={"session_id": "id_k14", "k": 14}
)

get_chat_history llamado con session_id=id_k14 y k=14


'Hola Javier, no olvides que ya hablamos sobre tus intereses en LangChain. ¿Quieres saber más sobre otras formas de memoria conversacional?'

In [76]:
chat_map["id_k14"].messages

[HumanMessage(content='hola! soy Javier', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hola Javier, ¿en qué puedo ayudarte hoy?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='cual es mi nombre?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hasta ahora solo sabemos que eres "Javier", pero no tengo información adicional sobre ti. ¿Quieres compartir algo más sobre ti mismo para personalizar mejor nuestra interacción?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Estoy investigando los distintos tipos de memoria conversacional en LangChain.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Qué interesante, ¿cuáles has estado explorando?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='He estado viendo ConversationBufferMemory y ConversationBufferWindowMemory.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Suena bien. ¿Cuál es la diferencia entre 

Listo! Hemos re'escrito uestro `BufferWindowMemory` usando el `RunnableWithMessageHistory` recomendado.

## 3.ConversationSummaryMemory

In [77]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(llm=llm)

C:\Users\Bo\AppData\Local\Temp\ipykernel_18808\988334424.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)


In [79]:
chain = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

In [80]:
chain.invoke({"input": "hola, mi nombre es Javier"})
chain.invoke({"input": "Estoy investigando los distintos tipos de memoria conversacional."})
chain.invoke({"input": "He estado viendo ConversationBufferMemory y ConversationBufferWindowMemory."})
chain.invoke({"input": "La memoria tipo buffer guarda toda la conversación completa."})
chain.invoke({"input": "La memoria tipo buffer window guarda solo los últimos k mensajes y descarta el resto."})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hola, mi nombre es Javier
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human introduces themselves as Javier, and the AI responds warmly, sharing its own identity and capabilities. The AI is designed to assist and converse with users like Javier, drawing from a vast amount of text data. The AI also notes that the human's name, Javier, has a personal touch, menti

{'input': 'La memoria tipo buffer window guarda solo los últimos k mensajes y descarta el resto.',
 'history': "Here is the updated summary:\n\nThe human introduces themselves as Javier, and the AI responds warmly, sharing its own identity and capabilities. The AI is designed to assist and converse with users like Javier, drawing from a vast amount of text data. The AI also notes that the human's name, Javier, has a personal touch, mentioning its meaning in Spanish culture.\n\nAdditionally, Javier shares a personal connection to the name, attributing it to their parents' love for music and Spanish culture.\n\nThe conversation then takes a turn to exploring conversational memory. Javier is investigating different types of memory related to conversations. The AI responds by explaining the various forms of memory that are relevant in this context, including Short-Term Memory (STM), Working Memory (WM), Long-Term Memory (LTM), and Episodic Memory.\n\nThe AI notes that each type of memory p

In [81]:
chain.invoke({"input": "cual es mi nombre?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Current summary:
The human introduces themselves as Javier, and the AI responds warmly, sharing its own identity and capabilities. The AI is designed to assist and converse with users like Javier, drawing from a vast amount of text data. The AI also notes that the human's name, Javier, has a personal touch, mentioning its meaning in Spanish culture.

Additionally, Javier shares a personal connection to the name, attributing it to their parents' love for music and Spanish culture.

The conversation then takes a turn to exploring conversational memory. Javier is investigating different types of memory related to conversations. The AI responds by explaining the various fo

{'input': 'cual es mi nombre?',
 'history': 'Current summary:\nThe human introduces themselves as Javier, and the AI responds warmly, sharing its own identity and capabilities. The AI is designed to assist and converse with users like Javier, drawing from a vast amount of text data. The AI also notes that the human\'s name, Javier, has a personal touch, mentioning its meaning in Spanish culture.\n\nAdditionally, Javier shares a personal connection to the name, attributing it to their parents\' love for music and Spanish culture.\n\nThe conversation then takes a turn to exploring conversational memory. Javier is investigating different types of memory related to conversations. The AI responds by explaining the various forms of memory that are relevant in this context, including Short-Term Memory (STM), Working Memory (WM), Long-Term Memory (LTM), and Episodic Memory.\n\nThe AI notes that each type of memory plays a crucial role in conversational AI like itself, and is happy to help Javi

## ConversationSummaryMemory with RunnableWithMessageHistory
Continuara...